In [ ]:
import cartopy
import cv2
import cmocean
import cftime
from datetime import datetime, timedelta
import numpy as np
import netCDF4 as nc
import pandas as pd
import pyproj as proj
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.dates as mdates
import metpy.calc as mpcalc
from scipy import io, interpolate, stats, signal
from scipy.optimize import curve_fit
from scipy.spatial import cKDTree

%matplotlib widget

In [ ]:
# Helping Functions
def datenum_to_datetime(datenum):
    """
    Convert Matlab datenum into Python datetime.
    :param datenum: Date in datenum format
    :return:        Datetime object corresponding to datenum.
    """
    days = datenum % 1
    return datetime.fromordinal(int(datenum)) \
           + timedelta(days=days) \
           - timedelta(days=366)

def latlon_to_local(lat, lon, lat_0, lon_0):
    crs_wgs = proj.Proj(init='epsg:4326')  # assuming you're using WGS84 geographic

    #Erect own local flat cartesian coordinate system
    cust = proj.Proj("+proj=aeqd +lat_0={0} +lon_0={1} +datum=WGS84 +units=m".format(lat_0, lon_0))
    x, y = proj.transform(crs_wgs, cust, lon, lat)
    return x, y


def local_to_latlon(x, y, lat_0, lon_0):
    # Define projections
    crs_wgs = proj.Proj(init='epsg:4326')  # WGS84 geographic
    cust = proj.Proj("+proj=aeqd +lat_0={0} +lon_0={1} +datum=WGS84 +units=m".format(lat_0, lon_0))

    # Transform from local AEQD projection back to geographic
    lon, lat = proj.transform(cust, crs_wgs, x, y)
    return lat, lon

def binned_statistics(x, y, bins=10, statistic='median'):
    """
    Bins the x variable into specified intervals, calculates the specified statistic (mean or median)
    of y values within each bin, and returns the statistic values, the bin centers, and the number
    of data points in each bin.

    Parameters:
    - x: array-like, continuous numerical variable to bin
    - y: array-like, continuous numerical variable for which the statistic is calculated within each bin
    - bins: int, the number of bins to create for the x variable
    - statistic: str, 'mean' or 'median' to specify which statistic to calculate

    Returns:
    - bin_centers: array, the center value of each bin
    - statistics: array, the calculated statistic (mean or median) for y values within each bin
    - counts: array, the number of data points in each bin
    """
    
    # Validate the statistic parameter
    if statistic not in ['mean', 'median', 'variance']:
        raise ValueError("Statistic must be 'mean', 'median', or 'variance'.")

    # Create bins
    bin_edges = np.linspace(np.min(x), np.max(x), bins + 1)
    binned_indices = np.digitize(x, bins=bin_edges)

    # Calculate the bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Calculate the specified statistic for each bin
    if statistic == 'mean':
        statistics = [np.nanmean(y[binned_indices == i]) for i in range(1, bins + 1)]
    elif statistic == 'median':
        statistics = [np.nanmedian(y[binned_indices == i]) for i in range(1, bins + 1)]
    elif statistic == 'variance':
        statistics = [np.nanvar(y[binned_indices == i]) for i in range(1, bins + 1)]
    
    # Compute std for each bin
    std_in_bin = [np.nanstd(y[binned_indices == i]) for i in range(1, bins + 1)]

    # Check Normal Distribution Goodness of Fit Test for each bin
    
    # Remove NaN values from the 

    # Count the number of data points for each bin
    counts = [np.sum(binned_indices == i) for i in range(1, bins + 1)]

    return np.array(bin_centers), np.array(statistics), np.array(counts), np.array(std_in_bin)

In [ ]:
# Add the wave gliders back in
df = pd.read_csv('../data/play1_df.csv')
df = df.dropna(subset=['hs'])
df.head()

# Load the Gridded Ice Maps
ice_map_data = io.loadmat('../data/L1/nws_2022.mat')
ice_map_lat = ice_map_data['LAT']
ice_map_lon = ice_map_data['LON']
ice_map_conc = ice_map_data['iceconc'] * 10
ice_map_datenum = np.squeeze(ice_map_data['date'])
ice_map_date = [datenum_to_datetime(ice_map_datenum[n].astype(np.float64)) for n in range(ice_map_datenum.size)]

# Find index of closest Ice Map - September 10th 
ind_for_ice_map = 252
print(ice_map_date[ind_for_ice_map])
ice_concentration = ice_map_conc[:,:,ind_for_ice_map]

# Compute Ice Edge Based on 80% Concentration
ice_conc_80percent = np.zeros(ice_concentration.shape)
ice_conc_80percent[ice_concentration >= 80] = 1

# Compute Ice Edge Based on 15% Concentration
ice_conc_15percent = np.zeros(ice_concentration.shape)
ice_conc_15percent[ice_concentration >= 15] = 1

# Compute Ice Edge Based on 0% Concentration
ice_conc_1percent = np.zeros(ice_concentration.shape)
ice_conc_1percent[ice_concentration >= 1] = 1

# Define the Local Cartesian Coordinate System
lat_0 = 72.48
lon_0 = -151.1

# Convert Lat lon on Ice Map to cartesian system 
x_icemap, y_icemap = latlon_to_local(ice_map_lat, ice_map_lon, lat_0, lon_0)

# Convert the SWIFT Track Coordinates to the local Cartesian system
x_swifts_gliders, y_swifts_gliders = latlon_to_local(df['latitude'], df['longitude'], lat_0, lon_0)

# Get time values for the SWIFTs
time_swifts = df['time'][:]
datetimes = pd.to_datetime(time_swifts)
date_numbers = np.squeeze(mdates.date2num(datetimes))

# Find the x and y location of the 15% ice concentration
# Find the 15% concentration contour line
ice_edge_contour_lon_vals = ice_map_lon[:,0]
lat_vals = ice_map_lat[0,:]

ice_edge_80percent_contour_lat_vals = []
ice_edge_15percent_contour_lat_vals = []
ice_edge_1percent_contour_lat_vals = []

for n in range(ice_conc_15percent.shape[0]):
    # 1% contour
    ice_edge_lat_index_array = np.where(ice_conc_1percent[n,:] == 1)[0]
    if ice_edge_lat_index_array.size > 0:
        ice_edge_1percent_contour_lat_vals.append(lat_vals[ice_edge_lat_index_array[0]])
    else:
        ice_edge_1percent_contour_lat_vals.append(np.NaN)
        
    # 15% contour
    ice_edge_lat_index_array = np.where(ice_conc_15percent[n,:] == 1)[0]
    if ice_edge_lat_index_array.size > 0:
        ice_edge_15percent_contour_lat_vals.append(lat_vals[ice_edge_lat_index_array[0]])
    else:
        ice_edge_15percent_contour_lat_vals.append(np.NaN)

    # 80% contour
    ice_edge_lat_index_array = np.where(ice_conc_80percent[n,:] == 1)[0]
    if ice_edge_lat_index_array.size > 0:
        ice_edge_80percent_contour_lat_vals.append(lat_vals[ice_edge_lat_index_array[0]])
    else:
        ice_edge_80percent_contour_lat_vals.append(np.NaN)

# Convert lon values to numpy array
ice_edge_80percent_contour_lat_vals = np.array(ice_edge_80percent_contour_lat_vals)
ice_edge_15percent_contour_lat_vals = np.array(ice_edge_15percent_contour_lat_vals)
ice_edge_1percent_contour_lat_vals = np.array(ice_edge_1percent_contour_lat_vals)


def smooth_ice_contour(lat_vals, lon_vals, lat_0, lon_0):
    # Convert the ice edge values to cartesian and polar coordinates
    x_iceedge, y_iceedge = latlon_to_local(lat_vals, lon_vals, lat_0, lon_0)

    # Interpolate the x and y ice edge values to make a smooth curve of the ice edge
    x_iceedge_interp = np.linspace(-100000, 200000, num=50000)
    y_iceedge_interp = np.interp(x_iceedge_interp, x_iceedge, y_iceedge)

    fs = 1 / (x_iceedge_interp[1] - x_iceedge_interp[0])
    cutoff = 1/40000
    order = 1
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y_iceedge_smoothed = signal.filtfilt(b, a, y_iceedge_interp)

    return x_iceedge_interp, y_iceedge_smoothed

# Smooth the ice concentration contours
x_80percent_contour, y_80percent_contour = smooth_ice_contour(ice_edge_80percent_contour_lat_vals, ice_edge_contour_lon_vals, lat_0, lon_0)
x_15percent_contour, y_15percent_contour = smooth_ice_contour(ice_edge_15percent_contour_lat_vals, ice_edge_contour_lon_vals, lat_0, lon_0)
x_1percent_contour, y_1percent_contour = smooth_ice_contour(ice_edge_1percent_contour_lat_vals, ice_edge_contour_lon_vals, lat_0, lon_0)


In [ ]:
# Compute the Cross Ice and Along Ice Coordinates for each point
def compute_cross_ice_distance(point_x, point_y, ice_x, ice_y):
    """
    Computes the signed normal (perpendicular) distance of an array of points from the ice edge defined by two arrays of x and y coordinates.
    The distance is negative if the ice edge is above the point (ice_y > point_y).
    
    Parameters:
        point_x (array-like): An array of x coordinates for the points.
        point_y (array-like): An array of y coordinates for the points.
        ice_x (array-like): An array of x coordinates defining the ice edge.
        ice_y (array-like): An array of y coordinates defining the ice edge.
    
    Returns:
        numpy.ndarray: An array of signed distances corresponding to each point.
    """
    points = np.column_stack((point_x, point_y))
    ice_points = np.column_stack((ice_x, ice_y))
    
    # Use KDTree to find the closest point on the ice edge
    tree = cKDTree(ice_points)
    distances, indices = tree.query(points)
    
    # Determine the sign of the distance based on the y-coordinate comparison
    signed_distances = np.where(ice_y[indices] > point_y, -distances, distances)
    
    return signed_distances, indices

cross_ice_distance_initial, indices = compute_cross_ice_distance(x_swifts_gliders, y_swifts_gliders, x_1percent_contour, y_1percent_contour)

fig, ax = plt.subplots()
ax.scatter(datetimes, cross_ice_distance_initial)